## PART 2: Direct and Global Separation

In [1]:
## Initialize everything

%run common.ipynb

Importing libraries...
Done!
Initialize camera and default settings...
Done!
Define functions needed to capture images...
Done!
Generating patterns for projector...
Done!


In [2]:
# We can change the brightness of our projector here.
# Try setting this value to some integer between 0 and 255.
projector_brightness = 255;

# Create a projection pattern
@widgets.interact_manual(brightness=widgets.IntSlider(min=0, max=255, step=1, value=255), 
                         pattern_id=widgets.Dropdown(
    options=[('All Black', 0), ('All White', 1), ('Horiz. Gradient', 2), ('Vert. Gradient', 3), ('Checkerboard', 4)],
    value=4,
    description='Pattern type',
))
def f(brightness, pattern_id):
    global projector_brightness;
    lightcrafter.set_pattern(brightness * patterns[:,:,pattern_id]);
    projector_brightness = brightness;
    time.sleep(1);

    image = numpy.double(get_color_image()) / 255.0;
    plt.imshow(image);
    plt.show();
    return;

aW50ZXJhY3RpdmUoY2hpbGRyZW49KEludFNsaWRlcih2YWx1ZT0yNTUsIGRlc2NyaXB0aW9uPXUnYnJpZ2h0bmVzcycsIG1heD0yNTUpLCBEcm9wZG93bihkZXNjcmlwdGlvbj11J1BhdHRlcm7igKY=


In [3]:
def update_progress(progress):
    bar_length = 20
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))

    clear_output(wait = True)
    text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)
    
    return;

In [4]:
min_buffer = 0;
max_buffer = 0;
background = 0;

# Capture and display raw image
class button_acquisition(widgets.Button):
    def on_button_clicked(b):
        ## Start acquisition phase
        global min_buffer, max_buffer, background;
        global button1;
        
        min_buffer = numpy.ones((100,160,3));
        max_buffer = numpy.zeros((100,160,3));

        lightcrafter.set_pattern(pattern_black);
        time.sleep(1);
        background = numpy.double(get_color_image()) / 255.0;

        itr = 0;
        step = 10;

        update_progress(itr);

        for i in range(0,2*num_pixels,step):
            for j in range(0,2*num_pixels,step):

                lightcrafter.set_pattern(numpy.roll(projector_brightness * pattern_checker,(i,j),(0,1)));
                time.sleep(1);
                buffer = numpy.double(get_color_image()) / 255.0;

                min_buffer = numpy.minimum(buffer, min_buffer);
                max_buffer = numpy.maximum(buffer, max_buffer);

                itr = itr + 1;
                update_progress(1.0*itr*step*step/(4*num_pixels*num_pixels))
        
        clear_output();
        display(button1)
                
                
button1 = button_acquisition(description="Start Acquisition")
button1.on_click(button_acquisition.on_button_clicked)

display(button1)

button_acquisition(description=u'Start Acquisition', style=ButtonStyle())

In [5]:
# Display captured data

class display_captured_data(widgets.Button):
    output = widgets.Output()
    
    @output.capture()
    def on_button_clicked(b):
        clear_output();
        
        print("Maximum pixel values")
        plt.imshow(max_buffer);
        plt.show();
        
        print("Minimum pixel values")
        plt.imshow(min_buffer);
        plt.show();
        
        print("Background (ambient) illumination")
        plt.imshow(background);
        plt.show();
        
button2 = display_captured_data(description="Display captured data")
button2.on_click(display_captured_data.on_button_clicked)

display(button2)
display(display_captured_data.output)

display_captured_data(description=u'Display captured data', style=ButtonStyle())

Output()

In [6]:
# Display direct and global separation results

class display_processed_data(widgets.Button):
    output = widgets.Output()
    
    @output.capture()
    def on_button_clicked(b):
        clear_output();
        direct_img = max_buffer-min_buffer;
        global_img = min_buffer-background;

        print("Direct component")
        plt.imshow(direct_img);
        plt.show();
        print("Global component")
        plt.imshow(global_img);
        plt.show();
        
button3 = display_processed_data(description="Display processed data")
button3.on_click(display_processed_data.on_button_clicked)

display(button3)
display(display_processed_data.output)

display_processed_data(description=u'Display processed data', style=ButtonStyle())

Output()